In [1]:
sc

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1575345745729_0001,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
<SparkContext master=yarn appName=livy-session-0>

In [2]:
bike_station_data = sc.textFile('s3://bigdata-desafio/lecture-replication-data/bike-share/stations.csv')
bike_status =  sc.textFile('s3://bigdata-desafio/lecture-replication-data/bike-share/status.csv')

In [3]:
bike_status.take(1)

['10,9,6,"2015-02-28 23:59:01"']

In [6]:
bike_station_proc = bike_station_data\
                    .map(lambda x: x.split(','))\
                    .map(lambda x: (int(x[0]),str(x[1]),float(x[2]),float(x[3]),int(x[4]),str(x[5]),str(x[6])))

In [8]:
bike_station_proc.take(3)

[(2, 'San Jose Diridon Caltrain Station', 37.329732, -121.901782, 27, 'San Jose', '8/6/2013'), (3, 'San Jose Civic Center', 37.330698, -121.888979, 15, 'San Jose', '8/5/2013'), (4, 'Santa Clara at Almaden', 37.333988, -121.894902, 11, 'San Jose', '8/6/2013')]

In [10]:
bike_status_proc = bike_status\
                    .map(lambda x: x.split(','))\
                    .map(lambda x: (x[0],x[1],x[2],x[3].replace('"','')))\
                    .map(lambda x: (x[0],x[1],x[2],x[3].split(' ')))\
                    .map(lambda x: (x[0],x[1],x[2],x[3][0].split('-'),x[3][1].split(':')))\
                    .map(lambda x: (int(x[0]),int(x[1]),int(x[2]),int(x[3][0]),int(x[3][1]),int(x[3][2]),int(x[4][0])))

In [11]:
bike_status_proc.take(1)

[(10, 9, 6, 2015, 2, 28, 23)]

In [16]:
get_station_by_city = bike_station_proc\
                        .map(lambda x:x[5])\
                        .map(lambda x: (x,1))\
                        .reduceByKey(lambda x,y : x+y)\
                        .sortBy(lambda x: x[1],ascending=False)

get_station_by_city_num = get_station_by_city.collect()

In [14]:
get_station_by_city.collect()

['San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'San Jose', 'Redwood City', 'Redwood City', 'Redwood City', 'Redwood City', 'Redwood City', 'Redwood City', 'Mountain View', 'Mountain View', 'Mountain View', 'Mountain View', 'Mountain View', 'Mountain View', 'Mountain View', 'Palo Alto', 'Palo Alto', 'Palo Alto', 'Palo Alto', 'Palo Alto', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Francisco', 'San Fr

In [17]:

get_station_by_city_num

[('San Francisco', 35), ('San Jose', 16), ('Redwood City', 7), ('Mountain View', 7), ('Palo Alto', 5)]

In [18]:
get_frisco_station_id = bike_station_proc\
                        .filter(lambda x: x[5] == 'San Francisco')\
                        .map(lambda x: (x[5],x[0]))\
                        .collect()
print(get_frisco_station_id)

[('San Francisco', 41), ('San Francisco', 42), ('San Francisco', 45), ('San Francisco', 46), ('San Francisco', 47), ('San Francisco', 48), ('San Francisco', 49), ('San Francisco', 50), ('San Francisco', 51), ('San Francisco', 39), ('San Francisco', 54), ('San Francisco', 55), ('San Francisco', 56), ('San Francisco', 57), ('San Francisco', 58), ('San Francisco', 59), ('San Francisco', 60), ('San Francisco', 61), ('San Francisco', 62), ('San Francisco', 63), ('San Francisco', 64), ('San Francisco', 65), ('San Francisco', 66), ('San Francisco', 67), ('San Francisco', 68), ('San Francisco', 69), ('San Francisco', 70), ('San Francisco', 71), ('San Francisco', 72), ('San Francisco', 73), ('San Francisco', 74), ('San Francisco', 75), ('San Francisco', 76), ('San Francisco', 77), ('San Francisco', 82)]

In [20]:
def get_frisco_id(x):
    if x >= 41 and x <= 82:
        return x

In [21]:
get_frisco_status = bike_status_proc\
                    .filter(lambda x: get_frisco_id(x[0]))

In [22]:
num = get_frisco_status.count()
num

453600

In [23]:
get_station_range = bike_station_proc\
                    .map(lambda x: (x[5],x[0]))\
                    .groupByKey()\
                    .map(lambda x: (x[0],list(x[1])))

In [24]:
get_station_range.collect()

[('Redwood City', [21, 22, 23, 24, 25, 26, 83]), ('Mountain View', [27, 28, 29, 30, 31, 32, 33]), ('San Francisco', [41, 42, 45, 46, 47, 48, 49, 50, 51, 39, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 82]), ('San Jose', [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 80, 84]), ('Palo Alto', [34, 35, 36, 37, 38])]

In [ ]:
for station in get_station_range.collect():
    tmp_key = bike_status_proc\
                .filter(lambda x: x[0] in station[1])

In [28]:
station_key_value = bike_station_proc.keyBy(lambda x: x[0])
status_key_value = bike_status_proc.keyBy(lambda x: x[0])

[(2, (2, 'San Jose Diridon Caltrain Station', 37.329732, -121.901782, 27, 'San Jose', '8/6/2013')), (3, (3, 'San Jose Civic Center', 37.330698, -121.888979, 15, 'San Jose', '8/5/2013')), (4, (4, 'Santa Clara at Almaden', 37.333988, -121.894902, 11, 'San Jose', '8/6/2013')), (5, (5, 'Adobe on Almaden', 37.331415, -121.8932, 19, 'San Jose', '8/5/2013')), (6, (6, 'San Pedro Square', 37.336721, -121.894074, 15, 'San Jose', '8/7/2013')), (7, (7, 'Paseo de San Antonio', 37.333798, -121.886943, 15, 'San Jose', '8/7/2013')), (8, (8, 'San Salvador at 1st', 37.330165, -121.885831, 15, 'San Jose', '8/5/2013')), (9, (9, 'Japantown', 37.348742, -121.894715, 15, 'San Jose', '8/5/2013')), (10, (10, 'San Jose City Hall', 37.337391, -121.886995, 15, 'San Jose', '8/6/2013')), (11, (11, 'MLK Library', 37.335885, -121.88566, 19, 'San Jose', '8/6/2013')), (12, (12, 'SJSU 4th at San Carlos', 37.332808, -121.883891, 19, 'San Jose', '8/7/2013')), (13, (13, 'St James Park', 37.339301, -121.889937, 15, 'San Jos

In [39]:
join_values = status_key_value.join(station_key_value)

In [40]:
join_values.take(1)

[(12, ((12, 10, 9, 2015, 2, 28, 23), (12, 'SJSU 4th at San Carlos', 37.332808, -121.883891, 19, 'San Jose', '8/7/2013')))]

In [41]:
clean_join = join_values.map( lambda x: (x[0],x[1][0][1],x[1][0][2],x[1][1][1]))

In [42]:
clean_join.take(2)

[(12, 10, 9, 'SJSU 4th at San Carlos'), (12, 10, 9, 'SJSU 4th at San Carlos')]

In [43]:
avg_hour = clean_join\
            .keyBy(lambda x:(x[3],x[2]))\
            .mapValues(lambda x: (x[1],1))\
            .reduceByKey(lambda x,y : (x[0]+y[0],x[1]+y[1]))\
            .mapValues(lambda x: x[0]/x[1])

In [44]:
avg_hour.take(2)

[(('SJSU 4th at San Carlos', 9), 9.973052138254248), (('SJSU 4th at San Carlos', 5), 13.597754548974061)]

In [49]:
avg_hour\
    .keyBy(lambda x: x[1])\
    .sortBy(lambda x: x[1],ascending=False)\
    .map(lambda x: (x[1][0][0],x[1][0][1],x[0]))\
    .take(10)

[('Yerba Buena Center of the Arts (3rd @ Howard)', 19, 0.0), ('Yerba Buena Center of the Arts (3rd @ Howard)', 18, 0.7314814814814815), ('Yerba Buena Center of the Arts (3rd @ Howard)', 17, 1.8636363636363635), ('Yerba Buena Center of the Arts (3rd @ Howard)', 16, 2.9351351351351354), ('Yerba Buena Center of the Arts (3rd @ Howard)', 15, 3.9548192771084336), ('Yerba Buena Center of the Arts (3rd @ Howard)', 14, 4.969216417910448), ('Yerba Buena Center of the Arts (3rd @ Howard)', 13, 5.938060309698452), ('Yerba Buena Center of the Arts (3rd @ Howard)', 12, 6.9878103837471786), ('Yerba Buena Center of the Arts (3rd @ Howard)', 11, 7.961840628507296), ('Yerba Buena Center of the Arts (3rd @ Howard)', 10, 8.876847290640393)]

In [46]:
avg_hour\
    .keyBy(lambda x: x[1]).take(1)

[(9.973052138254248, (('SJSU 4th at San Carlos', 9), 9.973052138254248))]

In [ ]:
s3://bigdata-desafio/challenges/u4lec1/household_power_consumption.txt

### Desafio
####  Moises Quiroz Diaz

### Ejercicio 1: Preliminares y datos perdidos

    Genere una instancia en AWS EMR y habilite un puerto dinámico para utilizar un notebook
    desde JupyterHub.
    Genere los objetos SparkConf y SparkContext .
    Utilizando Spark , importe el archivo household_power_consumption.txt a un objeto RDD.
    El primer registro del archivo corresponde a las etiquetas asociadas a cada columna. Elimínelo
    de su RDD.
    Cada campo dentro del registro está separado por un ; . Separe cada campo del registro por
    éste caracter.
    Reporte las siguientes métricas:

In [ ]:
sc #Se asume se autogeneran el conf y contex al estar trabajando dentro de aws con una instancia de spark

In [51]:
df = sc.textFile('s3://bigdata-desafio/challenges/u4lec1/household_power_consumption.txt')

In [62]:
header = df.first()
header = sc.parallelize([header])
df = df.subtract(header)

In [76]:
df_proc = df\
            .map(lambda x: x.split(';'))\
            .map(lambda x: (str(x[0]),str(x[1]),str(x[2]),str(x[3]),str(x[4]),str(x[5]),str(x[6]),str(x[7]),str(x[8])))


    La cantidad total de registros antes de eliminar los datos perdidos.
    La cantidad total de registros nulos.
    La cantidad de registros nulos por año. ¿En qué año hubo más registros nulos?
    La cantidad de registros nulos por mes para el año con una mayor cantidad de registros
    nulos. ¿En qué mes hubo más registros nulos?

In [79]:
print('la cantidad total de registros antes de eliminar es:',df_proc.count())

la cantidad total de registros antes de eliminar es: 2075259

In [88]:
nulos = df_proc.keyBy(lambda x: (x[0],x[1]))\
    .filter(lambda x: '?' in x[1])
print('la cantidad total de registros con datos nulos es:',nulos.count())

la cantidad total de registros con datos nulos es: 25979

In [91]:
perdidos_año = nulos\
                    .map(lambda x:x[0][0].split('/'))\
                    .map(lambda x: x[2])\
                    .map(lambda x: (x,1))\
                    .reduceByKey(lambda x,y : x+y)\
                    .sortBy(lambda x: x[1],ascending=False)

perdidos_año_num = perdidos_año.collect()

In [95]:
print('El año con mas perdidos fue el 2010','\n','*'*50)
for anio,cantidad in perdidos_año_num:
    print('Para el año',anio,'los datos perdidos fueron',cantidad)

El año con mas perdidos fue el 2010 
 **************************************************
Para el año 2010 los datos perdidos fueron 17629
Para el año 2009 los datos perdidos fueron 4280
Para el año 2007 los datos perdidos fueron 3931
Para el año 2008 los datos perdidos fueron 135
Para el año 2006 los datos perdidos fueron 4

In [101]:
get_nulos_2010 = nulos\
                    .filter(lambda x:x[0][0].split('/')[2] == '2010')\
                    .map(lambda x:x[0][0].split('/'))\
                    .map(lambda x: x[1])\
                    .map(lambda x: (x,1))\
                    .reduceByKey(lambda x,y : x+y)\
                    .sortBy(lambda x: x[1],ascending=False)

get_nulos_2010_num = get_nulos_2010.collect()

El año con mas perdidos fue el mes 8 
 **************************************************
Para el mes 2010 los datos perdidos fueron 17629
Para el mes 2009 los datos perdidos fueron 4280
Para el mes 2007 los datos perdidos fueron 3931
Para el mes 2008 los datos perdidos fueron 135
Para el mes 2006 los datos perdidos fueron 4

In [102]:
print('El año con mas perdidos fue el mes 8','\n','*'*50)
for mes,cantidad in get_nulos_2010_num:
    print('Para el mes',mes,'los datos perdidos fueron',cantidad)

El año con mas perdidos fue el mes 8 
 **************************************************
Para el mes 8 los datos perdidos fueron 7226
Para el mes 9 los datos perdidos fueron 5237
Para el mes 1 los datos perdidos fueron 3131
Para el mes 3 los datos perdidos fueron 2027
Para el mes 6 los datos perdidos fueron 2
Para el mes 2 los datos perdidos fueron 2
Para el mes 10 los datos perdidos fueron 1
Para el mes 4 los datos perdidos fueron 1
Para el mes 7 los datos perdidos fueron 1
Para el mes 5 los datos perdidos fueron 1

### Ejercicio 2: Preparación de los datos

Genere un objeto donde se encuentren sólo aquellos datos sin registros perdidos.

In [106]:
df_2 = df_proc.keyBy(lambda x: (x[0],x[1]))\
    .filter(lambda x: '?' not in x[1]).map(lambda x:x[1])

Con el objeto sin datos perdidos, genere un objeto RDD donde los registro tengan la siguiente
estructura:

In [107]:
df_2.take(1)

[('3/12/2008', '05:15:00', '0.272', '0.052', '245.030', '1.200', '0.000', '0.000', '0.000')]

In [111]:
df_proc_2 = df_2\
            .map(lambda x: (x[0].split('/'),x[2],x[3],x[4],x[5],x[6],x[7],x[8]))\
            .map(lambda x: (x[0][2],x[0][1],x[0][0],x[1],x[2],x[3],x[4],x[5],x[6],x[7]))\
            .map(lambda x: (int(x[0]),int(x[1]),int(x[2]),float(x[3]),float(x[4]),float(x[5]),float(x[6]),float(x[7]),float(x[8])))\
            .sortBy(lambda x: x[0],ascending=True)

print(df_proc_2.take(1))




[(2006, 12, 16, 3.668, 0.51, 233.99, 15.8, 0.0, 1.0)]

Devuelva con collect las primeras 5 observaciones

In [114]:
for i in df_proc_2.collect()[0:5]:
    print(i)

(2006, 12, 16, 3.666, 0.528, 235.68, 15.8, 0.0, 1.0)
(2006, 12, 16, 3.52, 0.522, 235.02, 15.0, 0.0, 2.0)
(2006, 12, 16, 3.702, 0.52, 235.09, 15.8, 0.0, 1.0)
(2006, 12, 16, 3.7, 0.52, 235.22, 15.8, 0.0, 1.0)
(2006, 12, 16, 4.448, 0.498, 232.86, 19.6, 0.0, 1.0)

### Ejercicio 3: Patrones globales

    Extraiga la media, desviación estándar e intervalos de confianza (a .5 desviaciones estándar)
    para las columnas global_activity_power , global_reactive_power , voltage y
    global_intensity .